찐 sasa

In [ ]:
!pip install --upgrade freesasa


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolfiles, rdmolops
import math, tempfile, os
import freesasa



# 사용자 제공 SMARTS 파일
csv_path = "cleaned_positive_fragments2.csv"
df = pd.read_csv(csv_path)

# max_sasa_dict 초기화
max_sasa_dict = {}
smarts_dict = {}
results = []

for i, row in df.iterrows():

    name = f"fg{i+1}"
    smarts = row["Fragment"]
    smarts_dict[name] = smarts

    try:
        frag = Chem.MolFromSmarts(smarts)
        if frag is None:
            results.append((name, smarts, "SMARTS 파싱 실패", None))
            continue
        # SMARTS → SMILES → Mol 변환
        smiles_from_smarts = Chem.MolToSmiles(frag, isomericSmiles=True)
        frag = Chem.MolFromSmiles(smiles_from_smarts)

        if frag is None:
            results.append((name, smarts, "SMARTS→SMILES 변환 실패", None))
            continue

        Chem.RemoveStereochemistry(frag)


        frag.UpdatePropertyCache(strict=False)
        rdmolops.SanitizeMol(frag, catchErrors=True)
        frag = Chem.AddHs(frag)

        params = AllChem.ETKDGv3()
        params.randomSeed = 42  # 재현성을 위해 시드 설정
        params.useRandomCoords = False #랜덤 좌표 off
        
        # 🔄 여러 개 conformer 생성
        conf_ids = AllChem.EmbedMultipleConfs(frag, numConfs=20, params=params)

        if not conf_ids:
            results.append((name, smarts, "3D 임베딩 실패", None))
            continue

        # 🔄 에너지 기준으로 최적의 conformer 선택
        best_energy = float('inf')
        best_conf_id = -1

        for conf_id in conf_ids:
            try:
                energy = AllChem.UFFOptimizeMolecule(frag, confId=conf_id)
                if energy < best_energy:
                    best_energy = energy
                    best_conf_id = conf_id
            except:
                continue

        if best_conf_id == -1:
            results.append((name, smarts, "UFF 최적화 실패", None))
            continue

        # 🔄 최적 conformer로부터 새 molecule 생성
        frag = Chem.Mol(frag, False, best_conf_id)

        conf = frag.GetConformer()
        if any(math.isnan(conf.GetAtomPosition(i).x) for i in range(frag.GetNumAtoms())):
            results.append((name, smarts, "NaN 좌표 존재", None))
            continue

        pdb_block = rdmolfiles.MolToPDBBlock(frag).replace("HETATM", "ATOM  ")
        pdb_block += "\nTER\nEND\n"

        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdb", mode='w') as tmp:
            pdb_path = tmp.name
            tmp.write(pdb_block)

        sasa_params = freesasa.Parameters()
        sasa_params.setAlgorithm(freesasa.LeeRichards)
        structure = freesasa.Structure(pdb_path)
        result = freesasa.calc(structure, sasa_params)

        max_sasa = round(result.totalArea(), 2)
        max_sasa_dict[name] = max_sasa
        results.append((name, smarts, "성공", max_sasa))


    except Exception as e:
        results.append((name, smarts, f"오류 발생: {str(e)}", None))
    finally:
        if 'pdb_path' in locals() and os.path.exists(pdb_path):
            os.remove(pdb_path)

max_sasa_dict, smarts_dict

# 결과 csv 파일로 저장
results_df = pd.DataFrame(results, columns=["Name", "SMARTS", "Status", "Max SASA"])
results_df.to_csv("positive_max_sasa_results_lee.csv", index=False)



positive fg의 MAX_SASA 계산 + relative_SASA 계산 + OUTSIDE/INSDIE 판별

In [ ]:
import os
import math
import tempfile
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolfiles
import freesasa

from collections import defaultdict



# 기능기별 INSIDE/OUTSIDE 누적 통계 저장용
stats = defaultdict(lambda: {"INSIDE": 0, "OUTSIDE": 0,"X": 0})
compound_hits = defaultdict(set)  # 기능기 이름 → set of SMILES 인덱스
threshold = 0.2  # 상대 SASA 기준값

# 🔧 기능기 목록을 smarts_dict에서 가져오도록 변경
SMARTS_LIST = smarts_dict

# SMILES 불러오기
df = pd.read_csv("final_data.csv")
smiles_list = df['smiles'].tolist()

# 결과 저장용 리스트
result_rows = []


for smiles in smiles_list:
    print(f"\n✅ SMILES: {smiles}")
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print("❌ MolFromSmiles 실패")
        continue

    # 수소 추가 및 3D 임베딩
    mol = Chem.AddHs(mol)
    status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    if status != 0:
        print("❌ 3D 임베딩 실패")
        continue

    try:
        AllChem.UFFOptimizeMolecule(mol)
    except Exception as e:
        print(f"❌ UFF 최적화 실패: {e}")
        continue

    conf = mol.GetConformer()
    if any(math.isnan(conf.GetAtomPosition(i).x) for i in range(mol.GetNumAtoms())):
        print("❌ NaN 좌표 존재")
        continue

    # PDB 저장 및 수정
    try:
        pdb_block = rdmolfiles.MolToPDBBlock(mol)

        # HETATM → ATOM 으로 수정 (freesasa 호환)
        pdb_block = pdb_block.replace("HETATM", "ATOM  ")

        # TER / END 추가
        if not pdb_block.endswith("\n"):
            pdb_block += "\n"
        pdb_block += "TER\nEND\n"

        # 임시 PDB 파일 저장
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdb", mode='w') as tmp:
            pdb_path = tmp.name
            tmp.write(pdb_block)

        # 🔍 저장된 PDB 확인
        with open(pdb_path, 'r') as f:
            print("🔍 저장된 PDB 파일 앞 10줄:")
            for line in f.readlines()[:10]:
                print(line.strip())

        # freesasa 구조 생성 및 계산
        sasa_params = freesasa.Parameters()
        sasa_params.setAlgorithm(freesasa.LeeRichards)
        structure = freesasa.Structure(pdb_path)
        result = freesasa.calc(structure, sasa_params)


        if structure.nAtoms() == 0:
            print("⚠️ 구조에 원자가 없습니다.")
            continue
        atom_sasa = [result.atomArea(i) for i in range(structure.nAtoms())]
        print(f"✅ SASA 계산 성공: 원자 수 = {structure.nAtoms()}")

    except Exception as e:
        print(f"❌ SASA 계산 실패: {e}")
        continue

    # 기능기별 relative SASA 계산
    for fg_name, smarts in SMARTS_LIST.items():
        pattern = Chem.MolFromSmarts(smarts)
        matches = mol.GetSubstructMatches(pattern)

        if not matches:
            print(f"⚠️ 기능기 매칭 실패: {fg_name}")
            continue

        compound_hits[fg_name].add(smiles)


        print(f"✅ 기능기 {fg_name} 매칭 개수: {len(matches)}")

        for match in matches:
            total_sasa = sum([atom_sasa[i] for i in match if i < len(atom_sasa)])
            max_sasa = max_sasa_dict.get(fg_name)

            # 안정망: max_sasa 없음 or 너무 작음    
            if max_sasa is None:
                print(f"⚠️ max_sasa 없음 → {fg_name} → 판정 제외")
                stats[fg_name]["X"] += 1
                continue
            elif max_sasa < 1e-3:
                print(f"⚠️ max_sasa 너무 작음 ({max_sasa:.4f}) → {fg_name} → 판정 제외")
                stats[fg_name]["X"] += 1
                continue


            rel_sasa = total_sasa / max_sasa
            print(f"🔸 {fg_name}의 relative SASA = {rel_sasa:.3f}")

            label = "INSIDE" if rel_sasa < threshold else "OUTSIDE"
            stats[fg_name][label] += 1
            print(f"   → {label}로 분류됨 (threshold={threshold})")

    # 임시 파일 삭제
    os.remove(pdb_path)


# ✅ 최종 결과 출력
print("\n📊 기능기별 INSIDE / OUTSIDE 비율:")
for fg, count in stats.items():
    total = count["INSIDE"] + count["OUTSIDE"] + count["X"]
    inside_pct = 100 * count["INSIDE"] / total if total else 0
    outside_pct = 100 * count["OUTSIDE"] / total if total else 0
    x_pct = 100 * count["X"] / total if total else 0
    print(f"{fg}: INSIDE {inside_pct:.1f}% ({count['INSIDE']}개), OUTSIDE {outside_pct:.1f}% ({count['OUTSIDE']}개), X {x_pct:.1f}% ({count['X']}개), 총 {total}개")



# 기능기별 요약 저장
summary_rows = []
for fg in SMARTS_LIST.keys():
    inside = stats[fg]["INSIDE"]
    outside = stats[fg]["OUTSIDE"]
    total = inside + outside
    inside_pct = 100 * inside / total if total else 0
    outside_pct = 100 * outside / total if total else 0
    compound_count = len(compound_hits[fg])  # 기능기를 포함한 고유 화합물 수

    summary_rows.append({
        "fg_name": fg,
        "fg_smarts": smarts_dict[fg],
        "INSIDE": inside,
        "OUTSIDE": outside,
        "INSIDE (%)": f"{inside_pct:.1f}",
        "OUTSIDE (%)": f"{outside_pct:.1f}",
        "X (%)": f"{x_pct:.1f}",
        "total_fg_count": total,
        "compound_count": compound_count,
        "majority_label":  (
            "X" if inside_pct == outside_pct == 0
            else "INSIDE" if inside_pct >= max(outside_pct, x_pct)
            else "OUTSIDE" if outside_pct >= max(inside_pct, x_pct)
            else "UNKNOWN"
        )
    })

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv("fg_summary_lee.csv", index=False)